In [18]:
#pip install yfinance pandas openpyxl

In [5]:
import os
import yfinance as yf
import pandas as pd
import datetime
from pathlib import Path

In [4]:
 os.getcwd()

'/home/deniskorsak/Desktop/Master Folder/Professional/03. Research/Research 2025/01. Code'

In [3]:
# --- Base project folder ---
base_dir = os.getcwd()

# --- Define subfolders ---
code_dir = os.path.join(base_dir, "01. Code")
data_dir = os.path.join(base_dir, "02. Data")

# --- Make sure folders exist ---
os.makedirs(code_dir, exist_ok=True)
os.makedirs(data_dir, exist_ok=True)

# --- Work from code folder ---
os.chdir(code_dir)

# --- Build filename with today's date ---
filename = f"eu_factor_data_{datetime.date.today()}.xlsx"
output_file = os.path.join(data_dir, filename)

print(f"Code running from: {code_dir}")
print(f"Data will be saved in: {output_file}")

NameError: name '__file__' is not defined

In [21]:
tickers = {
    # Equity Risk Premium [growth driver]
    "MSCI World (EUNL)": "EUNL.DE",   # Xetra listing
    "MSCI Europe (IMEU)": "IMEU.L",   # London listing
    
    # Duration Risk [LT diversifier]
    "Euro Gov Bonds 15-30yr (IBGL)": "IBGL.DE",
    "Euro Gov Bonds 10Y+ (EGIL)": "EGIL.PA",   # Paris listing
    
    # Credit Spread [credit risk]
    "Euro Corporate Bonds (IBCX)": "IBCX.DE",
    "Euro High Yield (IHYG)": "IHYG.L",
    
    # Inflation Hedge [real assets]
    "Euro Inflation Linked (IBCI)": "IBCI.DE",
    "Commodities (WCOB)": "WCOB.L"
}

In [22]:
df = pd.DataFrame()

In [23]:
for name, ticker in tickers.items():
    try:
        data = yf.Ticker(ticker).history(period="max")[["Close"]]
        data.index = data.index.tz_localize(None)
        data.rename(columns={"Close": name}, inplace=True)
        
        if df.empty:
            df = data
        else:
            df = df.join(data, how="outer")
    except Exception as e:
        print(f"Error downloading {name} ({ticker}): {e}")


$IBGL.DE: possibly delisted; no timezone found
$EGIL.PA: possibly delisted; no timezone found


Error downloading Euro Gov Bonds 15-30yr (IBGL) (IBGL.DE): 'Index' object has no attribute 'tz_localize'
Error downloading Euro Gov Bonds 10Y+ (EGIL) (EGIL.PA): 'Index' object has no attribute 'tz_localize'


In [24]:
# --- Export to Excel ---
df.to_excel(output_file, engine="openpyxl")